In [2]:
import xnat
import pydicom
from pydicom import dcmread
from pydicom.data import get_testdata_file
# import dicom
import zipfile
import os
import shutil


In [3]:
project_ID = "TEST_RETEST"
# dir = "\\\\uniwa.uwa.edu.au\\userhome\\staff3\\00105493\\Downloads\\MEL00033"
dir = "D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake"
user = "JulietLi"
password = ""
#Connect to XNAT session
# session = xnat.connect('http://aiqprdeauxnat001.australiaeast.cloudapp.azure.com:8080/xnat-web-1.8.5', user='admin', password='...')
session = xnat.connect('http://instance-146-118-67-25.nimbus.pawsey.org.au', user=user, password=password)


In [3]:
#Function to Zip Directory of Individual Patients
def zip_directory(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, mode='w') as zipf:
        len_dir_path = len(folder_path)
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, file_path[len_dir_path:])

In [4]:
#Remove Zips
for subdir in os.listdir(dir):
    if (".csv" not in subdir) and (".txt" not in subdir) and "." not in subdir:
        print("Now Processing: " + os.path.join(dir, subdir))
        for subFolder in os.listdir(os.path.join(dir, subdir)):
            if ".zip" in subFolder:
                print(os.path.join(dir, subdir,subFolder))
                os.remove(os.path.join(dir, subdir,subFolder))
           

Now Processing: D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-25 1
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-25 1\RP-25 1_RP-25_RTst_2021-09-24_155335_PET.NC.Research_Saved.Session..Universal.LesionID.-.ANZUP.(PSMA).2tp.done_n1__00000.zip
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-25 1\1.3.51.0_13128190_CT_2021-09-24_155335_PET.NC.Research_CT.Soft.Tissue.Whole.Body_n246__00000.zip
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-25 1\1.3.51.0_13128190_PT_2021-09-24_155335_PET.NC.Research_PET.WB.AC_n614__00000.zip
Now Processing: D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-02 2
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-02 2\RP-02 2_RP-02_RTst_2020-07-24_155128_PET.PSMA_Saved.session.2TP.final_n1__00000.zip
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-02 2\2.16.840_662.3912_CT_2020-07-24_155128_PET.PSMA_CT.Soft.Tissue.Whole.Body_n22

In [5]:
for subdir in os.listdir(dir):
    if (".csv" not in subdir) and (".txt" not in subdir) and "." not in subdir:
        print("Now Processing: " + os.path.join(dir, subdir))
        ct = False
        pt = False
        if not os.path.exists(os.path.join(dir,subdir,"_".join(subdir.split(' ')).replace("-","_"))):
            os.makedirs(os.path.join(dir,subdir,"_".join(subdir.split(' ')).replace("-","_")))
        for subFolder in os.listdir(os.path.join(dir, subdir)):
            if ("RTst" in subFolder) and os.path.isdir(os.path.join(dir, subdir,subFolder)):
                for file in os.listdir(os.path.join(dir, subdir,subFolder)):
                    if ".dcm" in file and file[0] != ".":
                        ds = pydicom.read_file(os.path.join(dir, subdir,subFolder,file))
                        if len(subdir.split(' ')) > 1:
                            pc_value = "Project:" + project_ID + " Subject:" + subdir.split(' ')[-2].replace("-","_") + " Session:" + "_".join(subdir.split(' ')).replace("-","_") + "_Rt"
                        else:
                            pc_value = "Project:" + project_ID + " Subject:" + subdir.split(' ')[-1].replace("-","_") + " Session:" + subdir.replace("-","_") + "_Rt"       
                        
                        if [0x0010, 0x4000] in ds:
                            ds[0x0010, 0x4000].value = pc_value
                        else:
                            ds.add_new([0x0010, 0x4000], 'LT', pc_value)
                        #save dicom file
                        ds.save_as(os.path.join(dir, subdir,subFolder,file))
                print("Zipping: ",os.path.join(dir, subdir,subFolder))
                zip_directory(os.path.join(dir, subdir,subFolder), os.path.join(dir, subdir,subFolder) + ".zip")
                print("Uploading:" + os.path.join(dir, subdir,subFolder) + ".zip")
                prearchive_session = session.services.import_(os.path.join(dir, subdir,subFolder) + ".zip", project=project_ID, destination='/prearchive')
            if ("CT" in subFolder or "PT" in subFolder) and os.path.isdir(os.path.join(dir, subdir,subFolder)):
                if "CT" in subFolder:
                    ct = True
                elif "PT" in subFolder:
                    pt = True

                for file in os.listdir(os.path.join(dir, subdir,subFolder)):
                    if ".dcm" in file and file[0] != ".":
                        ds = pydicom.read_file(os.path.join(dir, subdir,subFolder,file))
                        if len(subdir.split(' ')) > 1:
                            pc_value = "Project:" + project_ID + " Subject:" + subdir.split(' ')[-2].replace("-","_") + " Session:" + "_".join(subdir.split(' ')).replace("-","_")
                        else:
                            pc_value = "Project:" + project_ID + " Subject:" + subdir.split(' ')[-1].replace("-","_") + " Session:" + subdir.replace("-","_")
                        if [0x0010, 0x4000] in ds:
                            ds[0x0010, 0x4000].value = pc_value
                        else:
                            ds.add_new([0x0010, 0x4000], 'LT', pc_value)
                        #save dicom file
                        ds.save_as(os.path.join(dir, subdir,subFolder,file))
                        shutil.copy(os.path.join(dir, subdir,subFolder,file), os.path.join(dir, subdir,os.path.join(dir,subdir,"_".join(subdir.split(' ')).replace("-","_")),file))
                if pt and ct:
                    print("Zipping: ",os.path.join(dir, subdir,os.path.join(dir,subdir,"_".join(subdir.split(' ')).replace("-","_"))))
                    zip_directory(os.path.join(dir, subdir,os.path.join(dir,subdir,"_".join(subdir.split(' ')).replace("-","_"))), os.path.join(dir, subdir,os.path.join(dir,subdir,"_".join(subdir.split(' ')).replace("-","_"))) + ".zip")
                    print("Uploading:" + os.path.join(dir, subdir,os.path.join(dir,subdir,"_".join(subdir.split(' ')).replace("-","_"))) + ".zip")
                    prearchive_session = session.services.import_(os.path.join(dir, subdir,os.path.join(dir,subdir,"_".join(subdir.split(' ')).replace("-","_"))) + ".zip", project=project_ID, destination='/prearchive')

Now Processing: D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-25 1


[WARNING] Found unexpect content type, but assuming zip based on the extension


Zipping:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-25 1\RP-25 1_RP-25_RTst_2021-09-24_155335_PET.NC.Research_Saved.Session..Universal.LesionID.-.ANZUP.(PSMA).2tp.done_n1__00000
Uploading:D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-25 1\RP-25 1_RP-25_RTst_2021-09-24_155335_PET.NC.Research_Saved.Session..Universal.LesionID.-.ANZUP.(PSMA).2tp.done_n1__00000.zip
Zipping:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-25 1\RP_25_1


[WARNING] Found unexpect content type, but assuming zip based on the extension


Uploading:D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-25 1\RP_25_1.zip
Now Processing: D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-02 2


...

Uploading:D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-15 1\RP_15_1.zip
Now Processing: D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-13 2
Zipping:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-13 2\RP_13_2


[WARNING] Found unexpect content type, but assuming zip based on the extension


Uploading:D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-13 2\RP_13_2.zip


[WARNING] Found unexpect content type, but assuming zip based on the extension


Zipping:  D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-13 2\RP-13 2_RP-13_RTst_2020-09-25_161912_PET.PSMA_Saved.Session.2TP.complete_n1__00000
Uploading:D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-13 2\RP-13 2_RP-13_RTst_2020-09-25_161912_PET.PSMA_Saved.Session.2TP.complete_n1__00000.zip


In [ ]:
session.disconnect()